In [1]:
import sys
sys.path.append('../..')

In [2]:
# 加载数据集
from paddlenlp.datasets import load_dataset
# 飞桨的NLP库
import paddlenlp as ppnlp
# 数据预处理
from utils.utils import prepare_train_features, prepare_validation_features
# 偏函数应用
from functools import partial
# 评估SQuAD任务
from paddlenlp.metrics.squad import squad_evaluate, compute_prediction
# 集合类
import collections
# 操作时间的函数
import time
# 理JSON数据
import json

/home/bignet/anaconda3/envs/knowledgeEngineer/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
# 加载dureader_robust数据集的训练集和验证集
train_ds, dev_ds = ppnlp.datasets.load_dataset('dureader_robust', splits=('train', 'dev'))

# 循环打印前两个训练样本的相关信息
for idx in range(2):
    # 打印问题
    print(train_ds[idx]['question'])
    # 打印问题的上下文
    print(train_ds[idx]['context'])
    # 打印问题的答案
    print(train_ds[idx]['answers'])
    # 打印答案在上下文中的开始位置
    print(train_ds[idx]['answer_starts'])
    # 打印一个空行，为了在控制台输出中区分各个样本
    print()

仙剑奇侠传3第几集上天界
第35集雪见缓缓张开眼睛，景天又惊又喜之际，长卿和紫萱的仙船驶至，见众人无恙，也十分高兴。众人登船，用尽合力把自身的真气和水分输给她。雪见终于醒过来了，但却一脸木然，全无反应。众人向常胤求助，却发现人世界竟没有雪见的身世纪录。长卿询问清微的身世，清微语带双关说一切上了天界便有答案。长卿驾驶仙船，众人决定立马动身，往天界而去。众人来到一荒山，长卿指出，魔界和天界相连。由魔界进入通过神魔之井，便可登天。众人至魔界入口，仿若一黑色的蝙蝠洞，但始终无法进入。后来花楹发现只要有翅膀便能飞入。于是景天等人打下许多乌鸦，模仿重楼的翅膀，制作数对翅膀状巨物。刚佩戴在身，便被吸入洞口。众人摔落在地，抬头发现魔界守卫。景天和众魔套交情，自称和魔尊重楼相熟，众魔不理，打了起来。
['第35集']
[0]

燃气热水器哪个牌子好
选择燃气热水器时，一定要关注这几个问题：1、出水稳定性要好，不能出现忽热忽冷的现象2、快速到达设定的需求水温3、操作要智能、方便4、安全性要好，要装有安全报警装置 市场上燃气热水器品牌众多，购买时还需多加对比和仔细鉴别。方太今年主打的磁化恒温热水器在使用体验方面做了全面升级：9秒速热，可快速进入洗浴模式；水温持久稳定，不会出现忽热忽冷的现象，并通过水量伺服技术将出水温度精确控制在±0.5℃，可满足家里宝贝敏感肌肤洗护需求；配备CO和CH4双气体报警装置更安全（市场上一般多为CO单气体报警）。另外，这款热水器还有智能WIFI互联功能，只需下载个手机APP即可用手机远程操作热水器，实现精准调节水温，满足家人多样化的洗浴需求。当然方太的磁化恒温系列主要的是增加磁化功能，可以有效吸附水中的铁锈、铁屑等微小杂质，防止细菌滋生，使沐浴水质更洁净，长期使用磁化水沐浴更利于身体健康。
['方太']
[110]



In [4]:
# 设置模型名称
MODEL_NAME = "bert-base-chinese"
# 使用指定模型名称初始化tokenizer
tokenizer = ppnlp.transformers.BertTokenizer.from_pretrained(MODEL_NAME)

[2023-06-25 12:24:58,291] [    INFO] - Already cached /home/bignet/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2023-06-25 12:24:58,298] [    INFO] - tokenizer config file saved in /home/bignet/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2023-06-25 12:24:58,298] [    INFO] - Special tokens file saved in /home/bignet/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


In [5]:
# 设置最大序列长度
max_seq_length = 512
# 设置文档滑动窗口的步长
doc_stride = 128

# 创建训练集转换函数，使用部分参数和tokenizer
train_trans_func = partial(prepare_train_features,
                           max_seq_length=max_seq_length,
                           doc_stride=doc_stride,
                           tokenizer=tokenizer)

# 对训练集进行转换，并进行批处理
train_ds.map(train_trans_func, batched=True)

# 创建验证集转换函数，使用部分参数和tokenizer
dev_trans_func = partial(prepare_validation_features,
                         max_seq_length=max_seq_length,
                         doc_stride=doc_stride,
                         tokenizer=tokenizer)

# 对验证集进行转换，并进行批处理
dev_ds.map(dev_trans_func, batched=True)

In [6]:
# 对于每个索引进行循环两次
for idx in range(2):
    # 打印训练集中的input_ids
    print(train_ds[idx]['input_ids'])
    # 打印训练集中的token_type_ids
    print(train_ds[idx]['token_type_ids'])
    # 打印训练集中的overflow_to_sample
    print(train_ds[idx]['overflow_to_sample'])
    # 打印训练集中的offset_mapping
    print(train_ds[idx]['offset_mapping'])
    # 打印训练集中的start_positions
    print(train_ds[idx]['start_positions'])
    # 打印训练集中的end_positions
    print(train_ds[idx]['end_positions'])
    print()

[101, 7391, 5966, 2466, 6121, 6756, 6381, 2497, 811, 1525, 702, 4277, 2094, 1962, 102, 2972, 5773, 886, 4500, 8286, 6121, 6756, 6381, 2497, 811, 511, 6121, 6756, 6381, 2497, 811, 4638, 1962, 1776, 8024, 1357, 1104, 754, 6121, 6756, 6381, 2497, 811, 4638, 3029, 1008, 1928, 6981, 5390, 8024, 6981, 5390, 6632, 7770, 6632, 1962, 8024, 1086, 2218, 3221, 2595, 817, 3683, 511, 6121, 6756, 6381, 2497, 811, 6981, 5390, 7444, 6206, 8990, 8158, 8187, 6631, 7770, 3926, 3029, 1008, 1928, 3683, 6772, 1962, 8024, 6821, 3416, 2497, 1169, 6228, 7574, 3926, 3251, 2428, 7770, 511, 1086, 2218, 3221, 817, 3419, 8024, 2595, 817, 3683, 7770, 738, 3221, 1377, 809, 966, 2533, 5440, 5991, 4638, 511, 8286, 6121, 6756, 6381, 2497, 811, 2769, 886, 4500, 749, 671, 3667, 3198, 7313, 8024, 6230, 2533, 8286, 6121, 6756, 6381, 2497, 811, 3683, 6772, 1962, 2497, 2533, 2408, 6235, 3683, 6772, 1920, 8024, 2400, 684, 912, 2139, 2141, 2669, 8024, 817, 3419, 2798, 9600, 8024, 1762, 8286, 1555, 1814, 1377, 809, 743, 1168, 511

In [7]:
import paddle
from paddlenlp.data import Stack, Dict, Pad

batch_size = 8

# 创建训练集的分布式批量采样器
train_batch_sampler = paddle.io.DistributedBatchSampler(
    train_ds, batch_size=batch_size, shuffle=True)

# 定义训练集的批处理函数
train_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    "start_positions": Stack(dtype="int64"),
    "end_positions": Stack(dtype="int64")
}): fn(samples)

# 创建训练集的数据加载器
train_data_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_sampler=train_batch_sampler,
    collate_fn=train_batchify_fn,
    return_list=True)

# 创建验证集的批采样器
dev_batch_sampler = paddle.io.BatchSampler(
    dev_ds, batch_size=batch_size, shuffle=False)

# 定义验证集的批处理函数
dev_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
}): fn(samples)

# 创建验证集的数据加载器
dev_data_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_sampler=dev_batch_sampler,
    collate_fn=dev_batchify_fn,
    return_list=True)

In [8]:
for step, batch in enumerate(train_data_loader, start=1):

        input_ids, segment_ids, start_positions, end_positions = batch
        print(input_ids)
        break

Tensor(shape=[8, 181], dtype=int64, place=Place(gpu_pinned), stop_gradient=True,
       [[101 , 7391, 5966, ..., 678 , 511 , 102 ],
        [101 , 7391, 5966, ..., 678 , 511 , 102 ],
        [101 , 7391, 5966, ..., 678 , 511 , 102 ],
        ...,
        [101 , 7391, 5966, ..., 678 , 511 , 102 ],
        [101 , 7391, 5966, ..., 678 , 511 , 102 ],
        [101 , 7391, 5966, ..., 678 , 511 , 102 ]])


In [9]:
# 设置想要使用模型的名称
model = ppnlp.transformers.BertForQuestionAnswering.from_pretrained(MODEL_NAME)

[2023-06-25 12:25:56,610] [    INFO] - Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

W0625 12:25:56.854789 13446 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.0, Runtime API Version: 11.7
W0625 12:25:56.869946 13446 gpu_resources.cc:91] device: 0, cuDNN Version: 8.4.
[2023-06-25 12:25:57,158] [ WARNING] - Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_no

In [10]:
# 训练过程中的最大学习率
learning_rate = 3e-5
# 训练轮次
epochs = 1
# 学习率预热比例
warmup_proportion = 0.1
# 权重衰减系数，类似模型正则项策略，避免模型过拟合
weight_decay = 0.01

# 计算总的训练步数
num_training_steps = len(train_data_loader) * epochs

# 创建学习率调度器，使用线性衰减和预热
lr_scheduler = ppnlp.transformers.LinearDecayWithWarmup(learning_rate, num_training_steps, warmup_proportion)

# 生成需要进行权重衰减的参数名称列表，排除所有bias和LayerNorm参数
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 创建AdamW优化器，设置学习率、参数和权重衰减
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

In [11]:
class CrossEntropyLossForSQuAD(paddle.nn.Layer):
    def __init__(self):
        super(CrossEntropyLossForSQuAD, self).__init__()

    def forward(self, y, label):
        start_logits, end_logits = y   # both shape are [batch_size, seq_len]
        start_position, end_position = label
        start_position = paddle.unsqueeze(start_position, axis=-1)
        end_position = paddle.unsqueeze(end_position, axis=-1)

        # 计算起始位置的损失
        start_loss = paddle.nn.functional.softmax_with_cross_entropy(
            logits=start_logits, label=start_position, soft_label=False)
        start_loss = paddle.mean(start_loss)

        # 计算结束位置的损失
        end_loss = paddle.nn.functional.softmax_with_cross_entropy(
            logits=end_logits, label=end_position, soft_label=False)
        end_loss = paddle.mean(end_loss)

        # 求平均损失
        loss = (start_loss + end_loss) / 2
        return loss

In [18]:
learning_rate = 3e-5
from utils.utils import evaluate

criterion = CrossEntropyLossForSQuAD()
global_step = 0

# 遍历每个训练轮次
for epoch in range(1, epochs + 1):
    # 遍历训练集中的每个批次数据
    for step, batch in enumerate(train_data_loader, start=1):
        global_step += 1
        input_ids, segment_ids, start_positions, end_positions = batch
        logits = model(input_ids=input_ids, token_type_ids=segment_ids)
        loss = criterion(logits, (start_positions, end_positions))

        if global_step % 100 == 0:
            print("global step %d, epoch: %d, batch: %d, loss: %.5f" % (global_step, epoch, step, loss))

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()

    # # 在每个轮次结束后进行验证集评估
    # evaluate(model=model, data_loader=dev_data_loader)

# 保存模型和tokenizer
model.save_pretrained('./checkpoint')
tokenizer.save_pretrained('./checkpoint')

global step 100, epoch: 1, batch: 100, loss: 0.00001
global step 200, epoch: 1, batch: 200, loss: 0.00001
global step 300, epoch: 1, batch: 300, loss: 0.00001
global step 400, epoch: 1, batch: 400, loss: 0.00001
global step 500, epoch: 1, batch: 500, loss: 0.00001
global step 600, epoch: 1, batch: 600, loss: 0.00001
global step 700, epoch: 1, batch: 700, loss: 0.00001
global step 800, epoch: 1, batch: 800, loss: 0.00001
global step 900, epoch: 1, batch: 900, loss: 0.00001
global step 1000, epoch: 1, batch: 1000, loss: 0.00001
global step 1100, epoch: 1, batch: 1100, loss: 0.00001
global step 1200, epoch: 1, batch: 1200, loss: 0.00001
global step 1300, epoch: 1, batch: 1300, loss: 0.00001
global step 1400, epoch: 1, batch: 1400, loss: 0.00001
global step 1500, epoch: 1, batch: 1500, loss: 0.00001
global step 1600, epoch: 1, batch: 1600, loss: 0.00001
global step 1700, epoch: 1, batch: 1700, loss: 0.00001
global step 1800, epoch: 1, batch: 1800, loss: 0.00001
global step 1900, epoch: 1, 

[2023-06-25 13:07:28,223] [    INFO] - Configuration saved in ./checkpoint/config.json


global step 2200, epoch: 1, batch: 2200, loss: 0.00001


[2023-06-25 13:07:28,642] [    INFO] - tokenizer config file saved in ./checkpoint/tokenizer_config.json
[2023-06-25 13:07:28,643] [    INFO] - Special tokens file saved in ./checkpoint/special_tokens_map.json


('./checkpoint/tokenizer_config.json',
 './checkpoint/special_tokens_map.json',
 './checkpoint/added_tokens.json')

In [24]:
from utils.utils import evaluate
# 在每个轮次结束后进行验证集评估
evaluate(model=model, data_loader=dev_data_loader)

Processing example: 1000
time per 1000: 57.42130756378174


TypeError: list indices must be integers or slices, not str

In [28]:
# 特定目标的容器，以提供Python的内置通用容器dict、list、set、tuple的替代选择
import collections
# 与时间相关的功能
import time
# 使用JSON数据
import json
# PaddlePaddle深度学习平台
import paddle
# 用于评估squad（Question Answering）任务性能的函数
from paddlenlp.metrics.squad import squad_evaluate, compute_prediction

model=model
data_loader=dev_data_loader
is_test=False

# 将模型设置为评估模式，关闭Dropout和BatchNorm等
model.eval()

# 初始化start和end位置的预测结果列表
all_start_logits = []
all_end_logits = []
# 获取当前时间，用于计算每1000个样本的处理时间
tic_eval = time.time()

# 遍历数据集中的每个batch
for batch in data_loader:
    # 从batch中提取input_ids和token_type_ids
    input_ids, token_type_ids = batch
    # 将input_ids和token_type_ids输入模型，获取开始和结束位置的预测结果
    start_logits_tensor, end_logits_tensor = model(input_ids, token_type_ids)

    # 遍历每个样本的预测结果
    for idx in range(start_logits_tensor.shape[0]):
        # 每处理1000个样本，打印处理进度和处理时间
        if len(all_start_logits) % 1000 == 0 and len(all_start_logits):
            print("Processing example: %d" % len(all_start_logits))
            print('time per 1000:', time.time() - tic_eval)
            tic_eval = time.time()

        # 将每个样本的预测结果添加到预测结果列表中
        all_start_logits.append(start_logits_tensor.numpy()[idx])
        all_end_logits.append(end_logits_tensor.numpy()[idx])

print(data_loader.dataset.data[0])
# 使用预测结果生成最终的答案
all_predictions, _, _ = compute_prediction(
    data_loader.dataset.data, data_loader.dataset.new_data,
    (all_start_logits, all_end_logits), False, 20, 30)

# 如果是测试阶段
if is_test:
    # 将预测结果写入JSON文件中
    with open('prediction.json', "w", encoding='utf-8') as writer:
        writer.write(
            json.dumps(
                all_predictions, ensure_ascii=False, indent=4) + "\n")
else:
    # 如果不是测试阶段，则对预测结果进行评估
    squad_evaluate(
        examples=data_loader.dataset.data,
        preds=all_predictions,
        is_whitespace_splited=False)

# 打印前5个问题和预测的答案
count = 0
for example in data_loader.dataset.data:
    count += 1
    print()
    print('问题：', example['question'])
    print('原文：', ''.join(example['context']))
    print('答案：', all_predictions[example['id']])
    if count >= 5:
        break

# 将模型设置回训练模式
model.train()

Processing example: 1000
time per 1000: 3.9928367137908936
{'id': 'f1aafd23e0e12281770401586c8a4710', 'title': '', 'context': '爬行垫根据中间材料的不同可以分为:XPE爬行垫、EPE爬行垫、EVA爬行垫、PVC爬行垫;其中XPE爬行垫、EPE爬行垫都属于PE材料加保鲜膜复合而成,都是无异味的环保材料,但是XPE爬行垫是品质较好的爬行垫,韩国进口爬行垫都是这种爬行垫,而EPE爬行垫是国内厂家为了减低成本,使用EPE(珍珠棉)作为原料生产的一款爬行垫,该材料弹性差,易碎,开孔发泡防水性弱。EVA爬行垫、PVC爬行垫是用EVA或PVC作为原材料与保鲜膜复合的而成的爬行垫,或者把图案转印在原材料上,这两款爬行垫通常有异味,如果是图案转印的爬行垫,油墨外露容易脱落。 当时我儿子爬的时候,我们也买了垫子,但是始终有味。最后就没用了,铺的就的薄毯子让他爬。', 'question': '爬行垫什么材质的好', 'answers': ['XPE'], 'answer_starts': [-1]}


TypeError: list indices must be integers or slices, not str